# Exploração Numérica do Problema

Sumário  
Definição do Problema  
        ---A Classificação  
        ---Os atributos  
Exploração Numérica  
        ---Média, mediana e percentis  
        ---Média, mediana e percentis por classe  
        --- Gráficos  
Árvore de Decisão  
        ---Completa  
        ---Reduzida  

In [1]:
import pandas
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from statistics import mean
from numpy.linalg import norm
from matplotlib.colors import ListedColormap
import random

In [2]:
#importação da base de dados
file = "covtype.data"

#nome dos atributos
#estamos descartando os atributos que descrevem o tipo do solo
names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area_Rawah', 'Wilderness_Area_Neota', 'Wilderness_Area_Comanche', 'Wilderness_Area_Cache', 'Cover_Type']

#de 0 a 13 são os atributos listados acima, e 54 é a classificação
usecols = list(range(0, 14)) + [54]

#especifico o tipo de alguns parametros(os que não são simplesmente numéricos)
dtype = {'Cover_Type': 'category', 'Wilderness_Area_Rawah' : bool, 'Wilderness_Area_Neota' : bool, 'Wilderness_Area_Comanche' : bool, 'Wilderness_Area_Cache' : bool}

#lê o arquivo num pandas.dataframe
dataset = pandas.read_csv(file, header = None, usecols = usecols, names = names, dtype = dtype)

#adicionando uma coluna adicional para sintetizar os 4 boleanos que representam a Wilderness_area. 
#para uma única instância, somente um dos 4 booleanos pode ser verdadeiro, logo eles, em realidade, funcionam como uma categorização
new_column = pandas.Series([1 if dataset['Wilderness_Area_Rawah'][i] else 
                            2 if dataset['Wilderness_Area_Neota'][i] else
                            3 if dataset['Wilderness_Area_Comanche'][i] else
                            4 for i in range(len(dataset.index)) ], dtype="category")
#elimina as colunas reduzidas
dataset = dataset.drop(columns=['Wilderness_Area_Rawah', 'Wilderness_Area_Neota', 'Wilderness_Area_Comanche', 'Wilderness_Area_Cache'])
#insere nova coluna na posição 10
dataset.insert(loc = 10, column = 'Wilderness_Area', value = new_column)

#atualiza names para refletir a mudança acima
names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Cover_Type']

print("Dataframe com %d exemplares e %d atributos importado com sucesso" % (dataset.values.shape[0], dataset.values.shape[1]))

Dataframe com 581012 exemplares e 12 atributos importado com sucesso


In [12]:
#corr = dataset.corr()['Cover_Type'][dataset.corr()['Cover_Type'] < 1].abs()
#corr.sort(ascending = False)
#corr.head()

KeyError: 'Cover_Type'

In [ ]:
#atributos importados e seus tipos de dados
print(dataset.dtypes)

In [ ]:
#quantidade de exemplares por classificação
dataset.groupby('Cover_Type').size()

In [ ]:
#um exemplar da base de dados, para visualização
dataset.head(1)

In [ ]:
#sumário dos dados, geral
dataset.describe()

In [ ]:
#sumário dos dados, por classificação
gp = dataset.groupby('Cover_Type')
for name in names:
    print(name)
    display(gp[name].describe())

In [ ]:
#um histograma da altura, que se provou o mais importante atributo
dataset['Elevation'].hist()
plt.show()

In [ ]:
dataset['Cover_Type'].hist()
plt.show()

In [ ]:
#faz um scatter plot maneiro
#para cada classificação, seleciona N amostras aleatórias pra serem plotadas
seed = 7
N = 250
#markerTypes = {'1':'.', '2': 'x', '3': 'o', '4': '^', '5': '1', '6': '2', '7': '3'}
markerTypes = {'1':'.', '2': '.', '3': '.', '4': '.', '5': '.', '6': '.', '7': '.'}
points = []
markers = []
i = 0
gp = dataset.groupby('Cover_Type', sort = False)
samples = gp.apply(lambda x: x.sample(n = N, random_state = seed))

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Elevation', 'Slope'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Elevation', 'Horizontal_Distance_To_Roadways'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Elevation', 'Horizontal_Distance_To_Fire_Points'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Elevation', 'Elevation'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#faz um scatter plot maneiro
#só pega N amostras, sem pegar em igual numero de cada classificação
seed = 7
N = 250
#markerTypes = {'1':'.', '2': 'x', '3': 'o', '4': '^', '5': '1', '6': '2', '7': '3'}
markerTypes = {'1':'.', '2': '.', '3': '.', '4': '.', '5': '.', '6': '.', '7': '.'}
points = []
markers = []
i = 0
gp = dataset#.groupby('Cover_Type', sort = False)
samples = gp.apply(lambda x: x.sample(n = 7*N, random_state = seed))

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Elevation', 'Horizontal_Distance_To_Roadways'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Elevation', 'Horizontal_Distance_To_Fire_Points'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Elevation', 'Elevation'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Horizontal_Distance_To_Fire_Points', 'Horizontal_Distance_To_Roadways'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Aspect', 'Slope'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

In [ ]:
#plota grafico name1 x name2
name1, name2 = 'Elevation', 'Wilderness_Area'
i = 0
fig, ax = plt.subplots()
for cover_type, group in samples.groupby('Cover_Type'):
    
    plt.scatter(group[name1], group[name2], marker = markerTypes[cover_type])
    
ax.set(xlabel=name1, ylabel=name2,
       title=name1 + ' x ' + name2)
plt.show()

# Árvore de Decisão

In [7]:
#algumas funções auxiliares
def print_feature_importances(dtc):
    fi = dtc.feature_importances_
    for i in range(len(fi)):
        msg = "%s: %f" % (names[i], fi[i])
        print(msg)
       
def print_decision_tree_info(estimator, verbose = False):
    
    n_nodes = estimator.tree_.node_count
    children_left = estimator.tree_.children_left
    children_right = estimator.tree_.children_right
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]  # seed is the root node id and its parent depth
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth[node_id] = parent_depth + 1

        # If we have a test node
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True

    if(verbose): print("The binary tree structure has %s nodes and has "
          "the following tree structure:"
          % n_nodes)
    for i in range(n_nodes):
        if is_leaves[i]:
            if(verbose): print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
        else:
            if (verbose):print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
                  "node %s."
                  % (node_depth[i] * "\t",
                     i,
                     children_left[i],
                     feature[i],
                     threshold[i],
                     children_right[i],
                     ))
    print()
    
    print("Node count:", n_nodes)
    print("Leaf node count:", sum(b for b in is_leaves))
    print("Max node depth:", node_depth.max())

In [8]:
array = dataset.values
X = array[:, 0:11]
Y = array[:, 11]

#preprocessa dataset
#estava preprocessando, mas não fazia diferença
#X_scale_temp = preprocessing.scale(X[:, 0:10])
#X_scaled = np.append(X_scale_temp, X[:, 10].reshape(-1, 1), axis = 1)


validation_size = 0.2
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size = validation_size, random_state = seed)

In [9]:
#tentar descobrir o nivel de importancia dos atributos baseado na arvore de decisão
dtc = DecisionTreeClassifier(random_state = seed)
dtc.fit(X_train, Y_train)
dtc.score(X_validation, Y_validation)

0.9265681608908548

In [ ]:
#feature importance diz a importÇancia relativa de cada atributo
print_feature_importances(dtc)

In [ ]:
print_decision_tree_info(dtc)

In [ ]:
#novamente árvore de decisão, porém reduzida a tres attributos
X_3 = np.zeros((array.shape[0], 3))
X_3[:, 0] = dataset['Elevation']
X_3[:, 1] = dataset['Horizontal_Distance_To_Roadways']
X_3[:, 2] = dataset['Horizontal_Distance_To_Fire_Points']
Y = dataset.values[:, 11]
validation_size = 0.2
seed = 7
X_train_3, X_validation_3, Y_train_3, Y_validation_3 = model_selection.train_test_split(X_3, Y, test_size = validation_size, random_state = seed)




In [ ]:
dtc_3 = DecisionTreeClassifier(random_state = seed)
dtc_3.fit(X_train_3, Y_train_3)
dtc_3.score(X_validation_3, Y_validation_3)

In [ ]:
print_feature_importances(dtc_3)

In [ ]:
print_decision_tree_info(dtc_3)

In [ ]:
#limited decision tree
dtc_lim = DecisionTreeClassifier(random_state = seed, max_depth = 20)
dtc_lim.fit(X_train, Y_train)
dtc_lim.score(X_validation, Y_validation)

In [ ]:
#feature importance diz a importÇancia relativa de cada atributo
fi = dtc_lim.feature_importances_
for i in range(len(fi)):
    msg = "%s: %f" % (names[i], fi[i])
    print(msg)

In [ ]:
estimator = dtc_lim

n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

In [ ]:
print("Node count:", n_nodes)
print("Leaf node count:", sum(b for b in is_leaves))
print("Max node depth:", node_depth.max())

# KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train, Y_train)
knn.score(X_validation, Y_validation)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train_3, Y_train_3)
knn.score(X_validation_3, Y_validation_3)